# Summary

Experiment with parsing an old conv file. Eventually could allow resuming an old conv, although that may be out of scope for this round.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
from collections import defaultdict
from fuzzywuzzy import fuzz, process
from itertools import zip_longest
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    ConversationManager
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [6]:
conv = ConversationManager()

In [99]:
full_text = load('alexa/conversations/sam_altman__2022.06.05__00.29.22.txt')

In [120]:
name2turns = defaultdict(list)
prev_name = None
for row in full_text.splitlines()[2:]:
    if not row: continue
    if ': ' not in ' '.join(row.split(' ')[:4]): 
        print(row)
        turns = name2turns[prev_name]
        if not turns:
            raise RuntimeError(
                'Failed to parse conversation string. Trying to attach line '
                'to previous speaker turn but there is no previous turn.'
            )
        turns[-1] = turns[-1] + '\n\n' + row
        continue
    name, _, turn = row.partition(': ')
    name2turns[name].append(turn)
    prev_name = name

1.Virtual Reality – I think VR has the potential to change the way we interact with both the digital and physical world. For example, it could be used for training and education, allowing people to gain skills and knowledge in a more immersive and realistic way. 2. Blockchain – Blockchain has the potential to revolutionize how we store and manage data.
2. Be persistent
3. Believe in yourself
4. Stay focused
5. Be passionate
6. Be coachable
7. Never give up on your dreams


In [121]:
conv.start_conversation('Sam Altman')

{'persona': 'sam_altman',
 'gender': 'M',
 'img_path': '/Users/hmamin/jabberwocky/data/conversation_personas/sam_altman/profile.jpg',
 'img_url': 'https://upload.wikimedia.org/wikipedia/commons/a/a2/Sam_Altman_TechCrunch_SF_2019_Day_2_Oct_3_%28cropped%29.jpg',
 'nationality': 'American'}

In [122]:
conv.user_turns = list(name2turns['Harrison'])
conv.gpt3_turns = list(name2turns['Sam Altman'])

In [123]:
conv.full_conversation().replace('Me:', 'Harrison:') == full_text

True

In [131]:
a, b = name2turns

In [133]:
b

'Sam Altman'

In [124]:
fuzz.ratio(
    conv.full_conversation().replace('Me:', 'Harrison:'),
    full_text
)

100

In [125]:
for i, (t1, t2) in enumerate(zip_longest(
    conv.full_conversation().replace('Me:', 'Harrison:').splitlines(),
    full_text.splitlines()
)):
    if t1 != t2:
        print(i)
        print('T1:', t1)
        print('T2:', t2)
        print(spacer())

In [126]:
print(conv.full_conversation().replace('Me:', 'Harrison:'))

This is a conversation with Sam Altman. Samuel H. Altman (born April 22, 1985) is an American entrepreneur, investor, programmer, and blogger. He is the CEO of OpenAI and the former president of Y Combinator.

Harrison: what do you think you would be doing if you didn't have your current job

Sam Altman: If I didn't have my current job, I would probably be doing something in the technology industry. I'm passionate about technology and its potential to change the world, so I would likely be working on some sort of innovative project or company.

Harrison: I like the idea technology theory but it seems like in practice a lot of it is just damn to selling ads

Sam Altman: I think that's a fair criticism. A lot of technology does end up being used for advertising and other commercial purposes. But I believe that there is still a lot of potential for technology to be used for good, to make the world a better place. For example, OpenAI is working on developing artificial intelligence in a re

In [127]:
print(full_text)

This is a conversation with Sam Altman. Samuel H. Altman (born April 22, 1985) is an American entrepreneur, investor, programmer, and blogger. He is the CEO of OpenAI and the former president of Y Combinator.

Harrison: what do you think you would be doing if you didn't have your current job

Sam Altman: If I didn't have my current job, I would probably be doing something in the technology industry. I'm passionate about technology and its potential to change the world, so I would likely be working on some sort of innovative project or company.

Harrison: I like the idea technology theory but it seems like in practice a lot of it is just damn to selling ads

Sam Altman: I think that's a fair criticism. A lot of technology does end up being used for advertising and other commercial purposes. But I believe that there is still a lot of potential for technology to be used for good, to make the world a better place. For example, OpenAI is working on developing artificial intelligence in a re

## Test ported version

In [139]:
conv = ConversationManager()

In [140]:
full_text = load('alexa/conversations/sam_altman__2022.06.05__00.29.22.txt')

In [148]:
conv.start_conversation('Sam Altman', resume_text=full_text)

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:1865: UserWarning: "Me" is named "Harrison" in loaded conversation but self.me is "Me". We'll be sticking with "Me".
  )


{'persona': 'sam_altman',
 'gender': 'M',
 'img_path': '/Users/hmamin/jabberwocky/data/conversation_personas/sam_altman/profile.jpg',
 'img_url': 'https://upload.wikimedia.org/wikipedia/commons/a/a2/Sam_Altman_TechCrunch_SF_2019_Day_2_Oct_3_%28cropped%29.jpg',
 'nationality': 'American'}

In [150]:
conv.start_conversation('Robert Sapolsky', resume_text=full_text)

Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/sample_stream_response.pkl.
Object loaded from /Users/hmamin/jabberwocky/data/misc/gooseai_sample_responses.pkl.


/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:1865: UserWarning: "Me" is named "Harrison" in loaded conversation but self.me is "Me". We'll be sticking with "Me".
  )
/Users/hmamin/jabberwocky/lib/jabberwocky/openai_utils.py:1870: UserWarning: Conversation partner is named "Sam Altman" in loaded conversation but requested name is "Robert Sapolsky". We'll assume "Robert Sapolsky" is correct.
  f'sticking with "{name}".'


{'persona': 'robert_sapolsky',
 'gender': 'M',
 'img_path': '/Users/hmamin/jabberwocky/data/conversation_personas/robert_sapolsky/profile.jpg',
 'img_url': 'https://upload.wikimedia.org/wikipedia/commons/b/b1/Robert_Sapolsky-edited.jpg',
 'nationality': 'American'}